# Hyper parameter tunning

In [54]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from sklearn import preprocessing 
from sklearn.model_selection import train_test_split
from keras.layers import Dense, Dropout
from sklearn.preprocessing import MinMaxScaler
from keras.optimizers import Adam
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

### Dataset cleaning

In [55]:
cwd = os.getcwd()
test_set = pd.DataFrame(pd.read_csv(os.path.join(cwd, 'datasets/test_data.csv')))
dataset = pd.DataFrame(pd.read_csv(os.path.join(cwd, 'datasets/train_data.csv')))

In [56]:
dataset.head()

,ID,parents,has_nurs,form,children,housing,finance,social,health,app_status
0,1,usual,less_proper,complete,3,critical,convenient,problematic,not_recom,0
1,2,pretentious,very_crit,completed,1,convenient,inconv,nonprob,not_recom,0
2,3,pretentious,proper,incomplete,1,less_conv,convenient,slightly_prob,priority,1
3,4,great_pret,improper,complete,1,convenient,convenient,nonprob,recommended,1
4,5,great_pret,less_proper,completed,1,convenient,convenient,slightly_prob,priority,1


In [57]:
dataset.shape

(10368, 10)

In [58]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10368 entries, 0 to 10367
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   ID          10368 non-null  int64 
 1   parents     10368 non-null  object
 2   has_nurs    10368 non-null  object
 3   form        10368 non-null  object
 4   children    10368 non-null  object
 5   housing     10368 non-null  object
 6   finance     10368 non-null  object
 7   social      10368 non-null  object
 8   health      10368 non-null  object
 9   app_status  10368 non-null  int64 
dtypes: int64(2), object(8)
memory usage: 810.1+ KB


In [59]:
test_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2592 entries, 0 to 2591
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   ID        2592 non-null   int64 
 1   parents   2592 non-null   object
 2   has_nurs  2592 non-null   object
 3   form      2592 non-null   object
 4   children  2592 non-null   object
 5   housing   2592 non-null   object
 6   finance   2592 non-null   object
 7   social    2592 non-null   object
 8   health    2592 non-null   object
dtypes: int64(1), object(8)
memory usage: 182.4+ KB


In [60]:
dataset[dataset.isnull().any(axis=1)]    # any null records available?
del dataset['ID']
del test_set['ID']

In [61]:
# Encoding
### Label Encoding

label_encoder = preprocessing.LabelEncoder()

# Label Encoding for ordinal columns
def clean_ordinal(dataset):
    columns_ordinal = ['parents', 'has_nurs', 'children', 'housing', 'social', 'health', 'finance']

    for column in columns_ordinal:
        dataset[column] = label_encoder.fit_transform(dataset[column])
    return dataset

### One Hot Encoding

onehot_encoder = preprocessing.OneHotEncoder(drop='first')

# OneHotEncoding for nominal columns
def clean_nominal(dataset):
    columns_nominal = ['form']

    cleaned_dataset = dataset

    for column in columns_nominal:
        X = onehot_encoder.fit_transform(dataset[column].values.reshape(-1, 1)).toarray()
        # create dataframe from encoded data
        dataset_onehot = pd.DataFrame(X, columns = [column + '_' + str(i) for i in range(X.shape[1])]) 
        # update dataset -> cleaned_dataset
        cleaned_dataset = pd.concat([dataset_onehot, cleaned_dataset], axis=1)
        # remove encoded column from dataset
        del cleaned_dataset[column]
    return cleaned_dataset

In [62]:
# Test set cleaning
test_set = clean_ordinal(test_set)
cleaned_test_set = clean_nominal(test_set)
cleaned_test_set.head()

,form_0,form_1,form_2,parents,has_nurs,children,housing,finance,social,health
0,0.0,0.0,0.0,2,3,0,0,0,0,2
1,0.0,0.0,0.0,2,3,0,0,1,0,1
2,0.0,0.0,0.0,2,3,0,0,1,1,1
3,0.0,0.0,0.0,2,3,0,2,0,1,2
4,0.0,0.0,0.0,2,3,0,2,0,1,1


In [63]:
cleaned_dataset = clean_nominal(clean_ordinal(dataset))
cleaned_dataset.head()

,form_0,form_1,form_2,parents,has_nurs,children,housing,finance,social,health,app_status
0,0.0,0.0,0.0,2,2,2,1,0,1,0,0
1,1.0,0.0,0.0,1,4,0,0,1,0,0,0
2,0.0,0.0,1.0,1,3,0,2,0,2,1,1
3,0.0,0.0,0.0,0,1,0,0,0,0,2,1
4,1.0,0.0,0.0,0,2,0,0,0,2,1,1


In [64]:
x = cleaned_dataset.iloc[:, 0:-1]
y = cleaned_dataset.iloc[:, -1]

In [65]:
scaler = MinMaxScaler() 

x = scaler.fit_transform(x)

In [66]:
# splitting dataset to training and testing set
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)

### Classifier

In [67]:
def create_model(neurons=4, activation='relu', dropout=0.2, optimizer='adam', init_mode='uniform'):
    classifier = Sequential()
    
    classifier.add(Dense(neurons, input_dim=10, activation=activation, kernel_initializer=init_mode))
    classifier.add(Dropout(dropout))
    classifier.add(Dense(neurons, input_dim=10, activation=activation, kernel_initializer=init_mode))
    classifier.add(Dropout(dropout))
    classifier.add(Dense(neurons, input_dim=10, activation=activation, kernel_initializer=init_mode))
    classifier.add(Dropout(dropout))
    classifier.add(Dense(1, activation='sigmoid', kernel_initializer=init_mode))
    
    # compile model
    classifier.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    
    return classifier

### Optimizer

In [68]:
model = KerasClassifier(
    build_fn=create_model,
    epochs=50,
    batch_size=10,
    verbose=0
)

In [69]:
optimizer = [
    'SGD',
    'RMSprop',
    'Adagrad',
    'Adadelta',
    'Adam',
    'Adamax',
    'Nadam'
]

In [70]:
param_grid = dict(optimizer=optimizer)
param_grid

{'optimizer': ['SGD',
  'RMSprop',
  'Adagrad',
  'Adadelta',
  'Adam',
  'Adamax',
  'Nadam']}

In [71]:
grid = GridSearchCV(estimator=model,
                    param_grid=param_grid,
                    n_jobs=-1,
                    cv=5
                   )

In [72]:
grid_result = grid.fit(x, y)

In [73]:
print('Best: {} for {}'.format(grid_result.best_score_, grid_result.best_params_))

Best: 0.8375739336013794 for {'optimizer': 'RMSprop'}


In [74]:
means, stds, params = grid_result.cv_results_['mean_test_score'], grid_result.cv_results_['std_test_score'], grid_result.cv_results_['params']

In [75]:
for mean, stdev, param in zip(means, stds, params):
    print('{} | {} | : {}'.format(mean, stdev, param))

0.6456406235694885 | 0.0019727381052540315 | : {'optimizer': 'SGD'}
0.8375739336013794 | 0.03101532054497413 | : {'optimizer': 'RMSprop'}
0.6456406235694885 | 0.0019727381052540315 | : {'optimizer': 'Adagrad'}
0.6456406235694885 | 0.0019727381052540315 | : {'optimizer': 'Adadelta'}
0.8127965211868287 | 0.08297147113429255 | : {'optimizer': 'Adam'}
0.79968181848526 | 0.08375401175855203 | : {'optimizer': 'Adamax'}
0.6768846035003662 | 0.06054613870348309 | : {'optimizer': 'Nadam'}


### Batch size and Number of Epochs

In [76]:
model = KerasClassifier(build_fn=create_model, verbose=0)

In [77]:
# define the grid search parameters
batch_size = [10, 20, 40, 60]
epochs = [10, 50, 100, 200]

In [78]:
param_grid = dict(batch_size=batch_size, epochs=epochs)

In [79]:
grid = GridSearchCV(estimator=model,
                    param_grid=param_grid,
                    n_jobs=-1, cv=3
                   )

In [80]:
grid_result = grid.fit(x, y)

KeyboardInterrupt: 

In [ ]:
print('Best: {} for {}'.format(grid_result.best_score_, grid_result.best_params_))

In [ ]:
means, stds, params = grid_result.cv_results_['mean_test_score'], grid_result.cv_results_['std_test_score'], grid_result.cv_results_['params']

In [ ]:
for mean, stdev, param in zip(means, stds, params):
    print('{} | {} | : {}'.format(mean, stdev, param))

### Weight Tuning

In [ ]:
model = KerasClassifier(build_fn=create_model,
                        epochs=100,
                        batch_size=10,
                        verbose=0
                       )

In [ ]:
init_mode = ['uniform', 'lecun_uniform', 'normal', 'zero', 'glorot_normal', 'glorot_uniform', 'he_normal', 'he_uniform']

In [ ]:
param_grid = dict(init_mode=init_mode)

In [ ]:
grid = GridSearchCV(estimator=model,
                    param_grid=param_grid,
                    n_jobs=-1,
                    cv=3
                   )

In [ ]:
grid_result = grid.fit(x, y)

In [ ]:
print('Best: {} for {}'.format(grid_result.best_score_, grid_result.best_params_))

In [ ]:
means, stds, params = grid_result.cv_results_['mean_test_score'], grid_result.cv_results_['std_test_score'], grid_result.cv_results_['params']

In [ ]:
for mean, stdev, param in zip(means, stds, params):
    print('{} | {} | : {}'.format(mean, stdev, param))

### Activation function

In [ ]:
model = KerasClassifier(build_fn=create_model,
                        epochs=100,
                        batch_size=10,
                        verbose=0
                       )

In [ ]:
activation = ['softmax', 'softplus', 'softsign', 'relu', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear']

In [ ]:
param_grid = dict(activation=activation)

In [ ]:
grid = GridSearchCV(estimator=model,
                    param_grid=param_grid,
                    n_jobs=-1,
                    cv=3
                    )

In [ ]:
grid_result = grid.fit(x, y)

In [ ]:
print('Best: {} for {}'.format(grid_result.best_score_, grid_result.best_params_))

In [ ]:
means, stds, params = grid_result.cv_results_['mean_test_score'], grid_result.cv_results_['std_test_score'], grid_result.cv_results_['params']

In [ ]:
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

##### Number of Neurons

In [ ]:
model = KerasClassifier(build_fn=create_model,
                        epochs=100,
                        batch_size=10,
                        verbose=0
                       )

In [ ]:
neurons = [4, 8, 16, 32, 64, 128, 256, 512]

In [ ]:
param_grid = dict(neurons=neurons)

In [ ]:
grid = GridSearchCV(estimator=model,
                    param_grid=param_grid,
                    n_jobs=-1,
                    cv=3
                   )

In [ ]:
grid_result = grid.fit(x, y)

In [ ]:
print('Best: {} for {}'.format(grid_result.best_score_, grid_result.best_params_))

In [ ]:
means, stds, params = grid_result.cv_results_['mean_test_score'], grid_result.cv_results_['std_test_score'], grid_result.cv_results_['params']

In [ ]:
for mean, stdev, param in zip(means, stds, params):
    print('{} | {} | : {}'.format(mean, stdev, param))

### Drop out

In [ ]:
from keras.constraints import maxnorm
from keras.layers import Dropout

def create_model(dropout_rate=0.0, weight_constraint=0):
    classifier = Sequential()

    classifier.add(Dense(22, input_dim=7, activation='relu', kernel_initializer=init, kernel_constraint=maxnorm(weight_constraint)))
    classifier.add(Dropout(dropout_rate))
    classifier.add(Dense(22, input_dim=7, activation='relu', kernel_initializer=init, kernel_constraint=maxnorm(weight_constraint)))
    classifier.add(Dropout(dropout_rate))
    classifier.add(Dense(1, activation='sigmoid', kernel_initializer=init))
    # Compile model
    classifier.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    
    return classifier

In [ ]:
model = KerasClassifier(build_fn=create_model,
                        epochs=100,
                        batch_size=10,
                        verbose=0
                       )

In [ ]:
weight_constraint = [1, 2, 3, 4, 5]
dropout_rate = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

In [ ]:
param_grid = dict(dropout_rate=dropout_rate, weight_constraint=weight_constraint)

In [ ]:
grid = GridSearchCV(estimator=model,
                    param_grid=param_grid,
                    n_jobs=-1,
                    cv=3
                   )

In [ ]:
grid_result = grid.fit(x, y)

In [ ]:
print('Best: {} for {}'.format(grid_result.best_score_, grid_result.best_params_))

In [ ]:
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']

In [ ]:
for mean, stdev, param in zip(means, stds, params):
    print('{} | {} | : {}'.format(mean, stdev, param))